# Sandbox
For playing around during development.

In [1]:
%pylab inline
import pysd

print 'Running version:', pysd.__version__
print 'from relative path:', pysd.__file__ 
import pandas as pd

Populating the interactive namespace from numpy and matplotlib
Running version: 0.4.0
from relative path: pysd/__init__.pyc


In [3]:
model = pysd.read_vensim('tests/test-models/tests/subscript_subranges/test_subscript_subrange.mdl')

In [5]:
model.run(return_columns=['subrange_vals'])

ValueError: could not broadcast input array from shape (3) into shape (2)

In [2]:
#mdl_file = 'tests/old_tests/vensim/Teacup.mdl'
mdl_file = 'tests/test-models/tests/subscript_3d_arrays/test_subscript_3d_arrays.mdl'
#mdl_file = 'tests/test-models/tests/subscript_2d_arrays/test_subscript_2d_arrays.mdl'
#mdl_file = 'tests/test-models/tests/subscript_individually_defined_1_of_2d_arrays/subscript_individually_defined_1_of_2d_arrays.mdl'
#mdl_file = 'tests/test-models/tests/subscript_individually_defined_1d_arrays/subscript_individually_defined_1d_arrays.mdl'
mdl_file = '../mounir/pdatvensim8k_RW_v2_5.mdl'
model = pysd.read_vensim(mdl_file)

Value:  0  is not in dictionary  {'crew_type': 1}


VisitationError: ValueError: 0 is not in list

Parse tree:
<Node called "Flows" matching "Indirect Crew Costs per Year[R1,Enlisted]=47403.1 ~~|
Indirect Crew Costs per Year[R2,Enlisted]=47403.1 ~~|
Indirect Crew Costs per Year[R3,Enlisted]=47403.1 ~~|
Indirect Crew Costs per Year[R4,Enlisted]=48234.2 ~~|
Indirect Crew Costs per Year[R5,Enlisted]=50780.6 ~~|
Indirect Crew Costs per Year[R6,Enlisted]=54159.5 ~~|
Indirect Crew Costs per Year[R7,Enlisted]=55093.6 ~~|
Indirect Crew Costs per Year[R8,Enlisted]=57593.7 ~~|
Indirect Crew Costs per Year[R9,Enlisted]=57977.3 ~~|
Indirect Crew Costs per Year[R1,Officer]=64695.8 ~~|
Indirect Crew Costs per Year[R2,Officer]=63779.7 ~~|
Indirect Crew Costs per Year[R3,Officer]=66242 ~~|
Indirect Crew Costs per Year[R4,Officer]=67878.7 ~~|
Indirect Crew Costs per Year[R5,Officer]=69308.3 ~~|
Indirect Crew Costs per Year[R6,Officer]=67708.3 ~~|
Indirect Crew Costs per Year[R7,Officer]=67708.3 ~~|
Indirect Crew Costs per Year[R8,Officer]=67708.3 ~~|
In

In [ ]:
the get_array_info tries to understand the names of the array indices based upon what they contain. 

In [ ]:
pysd.functions.

In [3]:
%%time
model.run(flatten_subscripts=True)

CPU times: user 16 ms, sys: 1.83 ms, total: 17.8 ms
Wall time: 16.7 ms


,stock_a__entry_1__column_1__depth_1,stock_a__entry_1__column_1__depth_2,stock_a__entry_1__column_2__depth_1,stock_a__entry_1__column_2__depth_2,stock_a__entry_2__column_1__depth_1,stock_a__entry_2__column_1__depth_2,stock_a__entry_2__column_2__depth_1,stock_a__entry_2__column_2__depth_2,stock_a__entry_3__column_1__depth_1,stock_a__entry_3__column_1__depth_2,stock_a__entry_3__column_2__depth_1,stock_a__entry_3__column_2__depth_2
0,1.00,2.00,2.00,4.00,3.00,5.00,4.00,3.00,5.00,1.00,6.00,4.00
1,1.01,2.02,2.02,4.05,3.03,5.02,4.04,3.04,5.05,1.05,6.06,4.06
2,1.02,2.04,2.04,4.10,3.06,5.04,4.08,3.08,5.10,1.10,6.12,4.12
3,1.03,2.06,2.06,4.15,3.09,5.06,4.12,3.12,5.15,1.15,6.18,4.18
4,1.04,2.08,2.08,4.20,3.12,5.08,4.16,3.16,5.20,1.20,6.24,4.24
5,1.05,2.10,2.10,4.25,3.15,5.10,4.20,3.20,5.25,1.25,6.30,4.30
6,1.06,2.12,2.12,4.30,3.18,5.12,4.24,3.24,5.30,1.30,6.36,4.36
7,1.07,2.14,2.14,4.35,3.21,5.14,4.28,3.28,5.35,1.35,6.42,4.42
8,1.08,2.16,2.16,4.40,3.24,5.16,4.32,3.32,5.40,1.40,6.48,4.48
9,1.09,2.18,2.18,4.45,3.27,5.18,4.36,3.36,5.45,1.45,6.54,4.54


In [4]:
# for troubleshooting large models, just run everything once...
for key, func in model.components._dfuncs.iteritems():
    print func
    print func()


<function _dstock_a_dt at 0x10a1b2758>
[ 0.01  0.2   0.3 ]
